## Put EE Image Layers on a Leaflet Map

In [ ]:
import ee
import ipyleaflet

ee.Initialize()

In [ ]:
collection_id = 'COPERNICUS/S2'
cloud_id = 'COPERNICUS/S2_CLOUD_PROBABILITY'
start_datetime = '2020-04-01'
end_datetime = '2020-07-01'

In [ ]:
s2 = ee.ImageCollection(collection_id)
s2cloud = ee.ImageCollection(cloud_id)

In [ ]:
s2joined = ee.Join.saveFirst('cloud_mask').apply(primary=s2,
                                                 secondary=s2cloud,
                                                 condition=ee.Filter.equals(leftField='system:index',
                                                                            rightField='system:index'))

In [ ]:
s2combined = ee.ImageCollection(s2joined).map(lambda i: i.addBands(i.get('cloud_mask')))

In [ ]:
image = s2combined.filterDate(start_datetime, end_datetime).mean()

In [ ]:
rgb = {'min': [0, 0, 0], 'max': [3000, 3000, 3000], 'bands': ['B4', 'B3', 'B2']}
cloud = {'min': [0], 'max': [100], 'bands': ['probability']}

In [ ]:
rgb_id = image.getMapId(rgb)

In [ ]:
rgb_id

In [ ]:
m = ipyleaflet.Map()

In [ ]:
m

In [ ]:
layer = ipyleaflet.TileLayer(url=rgb_id['tile_fetcher'].url_format,
                             name='S2 RGB')

In [ ]:
m.add_layer(layer)

In [ ]:
cloud_id = image.getMapId(cloud)
print(cloud_id)

In [ ]:
cloud_layer = ipyleaflet.TileLayer(url=cloud_id['tile_fetcher'].url_format,
                             name='S2 Cloud')

In [ ]:
m.add_layer(cloud_layer)